In [27]:
import pandas as pd

In [37]:
# Sample data
data = {
        'chr': ['chr1', 'chr1', 'chr1', 'chr1'],
    'strand': ['-', '-', '+', '+'],
    'Acceptor_start': [612825, 612825, 1043188, 1043188],
    'Acceptor_end': [612835, 612835, 1043198, 1043198],
    'varinat_start': [612826, 612830, 1043190, 1043195],
    'variant_end': [612827, 612831, 1043191, 1043196],
    'ref_neucleotide': ['T', 'C', 'G', 'A'],
    'alternative_neucleotide': ['GC', 'T', 'T', 'G'],
    'reference_seq': ['ATAAACAAAAA', 'ATAAACAAAAA', 'CCGCCCCACCC', 'CCGCCCCACCC']
}
# Sample data
data = {
    'chr': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'],
    'strand': ['-', '-', '+', '+', '+'],
    'Acceptor_start': [612825, 612825, 1043188, 1043188, 1043188],
    'Acceptor_end': [612835, 612835, 1043198, 1043198, 1043198],
    'varinat_start': [612826, 612830, 1043190, 1043195, 1043192],
    'variant_end': [612827, 612831, 1043191, 1043196, 1043194],
    'ref_neucleotide': ['T', 'C', 'G', 'A', 'TG'],
    'alternative_neucleotide': ['GC', 'T', 'T', 'G', 'G'],
    'reference_seq': ['ATAAACAAAAA', 'ATAAACAAAAA', 'CCGCTGCACCC', 'CCGCTGCACCC', 'CCGCTGCACCC']
}

In [39]:
df = pd.DataFrame(data)
df

,chr,strand,Acceptor_start,Acceptor_end,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,reference_seq
0,chr1,-,612825,612835,612826,612827,T,GC,ATAAACAAAAA
1,chr1,-,612825,612835,612830,612831,C,T,ATAAACAAAAA
2,chr1,+,1043188,1043198,1043190,1043191,G,T,CCGCTGCACCC
3,chr1,+,1043188,1043198,1043195,1043196,A,G,CCGCTGCACCC
4,chr1,+,1043188,1043198,1043192,1043194,TG,G,CCGCTGCACCC


In [40]:
# Function to aggregate data for unique acceptor regions
def aggregate_variants(group):
    group_data = {
        'varinat_start': [group['varinat_start'].tolist()],
        'variant_end': [group['variant_end'].tolist()],
        'ref_neucleotide': [group['ref_neucleotide'].tolist()],
        'alternative_neucleotide': [group['alternative_neucleotide'].tolist()],
        'alt_seq': modify_sequence(group)
    }
    return pd.Series(group_data, index=['varinat_start', 'variant_end', 'ref_neucleotide', 'alternative_neucleotide', 'alt_seq'])

In [41]:
# Function to modify the sequence based on variants
def modify_sequence(group):
    reference_sequence = list(group['reference_seq'].iloc[0])  # Assuming same reference for each group
    print(reference_sequence)
    adjustments = 0  # To account for the shifts due to insertions/deletions
    for _, variant in group.sort_values('varinat_start').iterrows():
        # Calculate the relative positions in the reference
        start_index = variant['varinat_start'] - group['Acceptor_start'].iloc[0] + adjustments
        end_index = variant['variant_end'] - group['Acceptor_start'].iloc[0] + adjustments
        # Calculate adjustment
        ref_len = end_index - start_index
        alt_len = len(variant['alternative_neucleotide'])
        # Perform the insertion, deletion, or substitution
        reference_sequence[start_index:end_index] = list(variant['alternative_neucleotide'])
        print(reference_sequence)
        # Adjust subsequent indices
        adjustments += (alt_len - ref_len)
    
    return ''.join(reference_sequence)

In [42]:
# Group by acceptor start and end, then aggregate variants
grouped_df = df.groupby(['chr', 'strand', 'Acceptor_start', 'Acceptor_end', 'reference_seq']).apply(aggregate_variants).reset_index()
grouped_df

['C', 'C', 'G', 'C', 'T', 'G', 'C', 'A', 'C', 'C', 'C']
['C', 'C', 'T', 'C', 'T', 'G', 'C', 'A', 'C', 'C', 'C']
['C', 'C', 'T', 'C', 'G', 'C', 'A', 'C', 'C', 'C']
['C', 'C', 'T', 'C', 'G', 'C', 'G', 'C', 'C', 'C']
['A', 'T', 'A', 'A', 'A', 'C', 'A', 'A', 'A', 'A', 'A']
['A', 'G', 'C', 'A', 'A', 'A', 'C', 'A', 'A', 'A', 'A', 'A']
['A', 'G', 'C', 'A', 'A', 'A', 'T', 'A', 'A', 'A', 'A', 'A']


,chr,strand,Acceptor_start,Acceptor_end,reference_seq,varinat_start,variant_end,ref_neucleotide,alternative_neucleotide,alt_seq
0,chr1,+,1043188,1043198,CCGCTGCACCC,"[[1043190, 1043195, 1043192]]","[[1043191, 1043196, 1043194]]","[[G, A, TG]]","[[T, G, G]]",CCTCGCGCCC
1,chr1,-,612825,612835,ATAAACAAAAA,"[[612826, 612830]]","[[612827, 612831]]","[[T, C]]","[[GC, T]]",AGCAAATAAAAA
